In [12]:
'''
new version, correct the way of fps calculation, the same result as the older one, but faster
input: file with empty/nonempty road and latency for each frame
output: file(at folder of "fps_converted") with accumulative "empty road %" and fps at the time of each frame

'''
import pandas as pd
import os
import numpy as np
import re
import random

random.seed(42)
################## choose files to convert here ##############
file = 'performance_optmized_yolov5_odCuda+erdCuda+lstmCuda_202310271823_4p.csv' ##YOLO

#  by selecting True or false to cover 0%-100% #####
ascending_flag = False ## True ->1, False: 2
ascending_flag = True ## True ->1, False: 2
##############################################################

data = pd.DataFrame(columns = ['nonempty_road','latency_whole'])
# data_road_status = pd.read_csv('../result/v0_road_status.csv')
data_perform = pd.read_csv(os.path.join('../result/raw_data',file))

# status_list= data_perform.nonempty_road
# updated_status_list = interp_status(status_list) ## interpolate -1 ro 0 or 1
data.nonempty_road = data_perform.nonempty_road
data.latency_whole = data_perform.latency_whole  ##all_cuda_latency  #latency_whole,

file_prefix = re.findall(r'(.+).csv',file)[0]
data = data[100:]  ## take out the gpu warm up 



data_0 = data.where(data.nonempty_road ==0).dropna()
data_1 = data.where(data.nonempty_road ==1).dropna()
data_0 = data_0.sort_values(by=['latency_whole'], ascending= ascending_flag)[5:-5] ## remove possible outliers
data_1 = data_1.sort_values(by=['latency_whole'], ascending= ascending_flag)[5:-5]
if ascending_flag:
    data = pd.concat([data_1,data_0])
else:
    data = pd.concat([data_0,data_1])

    # data = data.sort_values(by=['empty_road_detection_result','latency_whole'], ascending=False) ## run twice by True and ascending=False
# data = data.iloc[:]  

data['nonempty_road']= data['nonempty_road'].astype(int) ### value 0 means road is empty
data['latency_whole']=data['latency_whole'].astype(float)
er_sum = 0
er_list = data['nonempty_road'].tolist()
latency_list = data['latency_whole'].tolist()

# accum_ave_latency = []
fps_list = []
empty_rate_list, accum_ave_latency, fps_list  = [], [], []

for cnt in range(1,len(er_list)):
    latency_total = sum(latency_list[:cnt])
    fps_list.append(cnt/latency_total)
    er_sum = sum(er_list[:cnt]) ## 0 means road is empty
    empty_rate_list.append(1-er_sum/cnt) 


result = pd.DataFrame(columns = ['road_empty %', 'fps'])
result['road_empty %'] = empty_rate_list
result['fps'] = fps_list

suffix = 'a' if ascending_flag == True  else 'd'
result.to_csv('../result/raw_data/'+'fps_result_'+file_prefix.split('_')[2]+'_'+suffix+'.csv')